# TP3 : Fenêtres de Parzen

Maintenant que vous êtes des experts en estimation de gaussiennes, on va découvrir une nouvelle technique pour la classification : les fenêtres de Parzen.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas

# Import de vos fonctions
...

Contrairement aux estimations de gaussiennes, on fait ici aucune supposition sur la distribution des données. On considère juste que chaque classe a une densité de probabilité que respecte les échantillons de la vérité terrain. L'estimation de cette densité de probabilité se fait justement grâce à ces échantillons. A chacun de ces échantillons est attribué un noyau, c'est-à-dire une fonction prédifinie qui fait office de densité de probabilité. Ce noyau est le même pour tous les échantillons. La densité de probabilité de la classe devient donc la moyenne de ce noyau appliqué sur chacun des échantillons de la classe. La formule de tout ce blabla est la suivante : 

$f(x) = \frac{1}{n} \sum_{i} \phi(x-x_{i})$

Dans cette magnifique formule, n est le nombre d'échantillons de la classe, $\phi$ est la fonction noyau, $x_{i}$ est l'échantillon i de la classe, et x est l'échantillon à prédire. Pour le point de vue graphique, faut venir en TP (chaud les graphiques sur Jupyter...)

**La question préliminaire :** Alors, les fenêtres de Parzen ? Méthode paramétrique ou non paramétrique ?

**_Réponse :_** 

## Partie 1 : Création des noyaux

On va d'abord créer les fonctions de noyau. Ces fonctions prendront en entrée l'échantillon à prédire, un échantillon d'une classe, et un paramètre h.

**1)** Commençons avec le noyau uniforme ! Le principe est ultra simple : on crée une densité de probabilité uniforme de taille hxh centrée sur l'échantillon de la classe (on est ici dans le cas 2D). Si notre échantillon à prédire se trouve dans cette densité de probabilité (donc dans le carré hxh), la fonction retourne $\frac{1}{h^{2}}$, sinon la fonction retourne 0. Créez cette fonction pour commencer.

In [ ]:
# A compléter
# Fonction de noyau uniforme
...

Testez votre fonction maintenant avec ces quelques tests unitaires réalisés par mes soins :D

In [ ]:
# Test 1 : L'échantillon est dans le noyau
A = np.array([2.5,3.6])
B = np.array([2.8,2.9])
h = 2
val_noyau_uniforme = ...
assert val_noyau_uniforme == 0.25, "Test 1 non fonctionnel"

# Test 2 : L'échantillon n'est pas dans le noyau
A = np.array([2.5,3.6])
B = np.array([3.8,2.9])
h = 1
val_noyau_uniforme = ...
assert val_noyau_uniforme == 0, "Test 2 non fonctionnel"

**2)** Créons un 2ème type de noyau : le noyau gaussien. Il n'est pas aussi simple que l'uniforme, mais ça va encore ;) Ici, comme son nom l'indique, la densité de probabilité est une fonction gaussienne 2D centré sur l'échantillon de la classe et de variance h. La formule du noyau est donc la suivante : 

$\phi(x,x_{i}) = \frac{1}{2\pi\sigma^{2}} e^{\frac{||x-x_{i}||^{2}}{2\sigma^{2}}}$

Ici, $\sigma$ est la variance, donc h, et x_{i} un échantillon de la classe. 

Là où on va se simplifier la vie, c'est qu'on n'a pas besoin de la "réelle" valeur de la distance, mais juste une valeur pour comparaison (on veut juste attribuer une classe à notre échantillon à prédire, pas obtenir une distance). On va donc garder que la partie exponentielle de la formule ci-dessus.

Bon, vous avez toutes les infos, donc créez cette fonction.

In [ ]:
# A compléter
# Fonction de noyau gaussien
...

Hop hop hop ! On teste la fonction avant la prochaine étape !

In [ ]:
# Test 1
A = np.array([2.5,3.6])
B = np.array([2.8,2.9])
h = 2
val_noyau_gaussien = ...
assert np.abs(val_noyau_gaussien-0.93)<0.01, "Test 1 non fonctionnel"

# Test 2
A = np.array([2.5,3.6])
B = np.array([3.8,2.9])
h = 1
val_noyau_gaussien = ...
assert np.abs(val_noyau_gaussien-0.33)<0.01, "Test 2 non fonctionnel"

## Partie 2 : Construction du modèle par fenêtres de Parzen

**1)** Créez le classifieur par fenêtres de Parzen, une classe python qui hérite de votre classe Modele de base. Dans le constructeur, vous aurez deux nouveaux paramètres : votre fonction de noyau (uniforme ou gaussien), et un paramètre h. La fonction de prédiction calcule la probabilité de l'échantillon à prédire pour chaque classe, qui est la moyenne (ou somme) des densités de probas des noyaux appliqués à chacun des échantillons d'apprentissage de chaque classe.

*Note :* La fonction d'apprentissage ici n'est pas vraiment intéressante, puisque il n'y a "aucun apprentissage". Maintenant, de mon côté, j'ai utilisé cette fonction pour stocker les échantillons d'apprentissage divisé par classes.

*Note 2 :*  Il se peut que votre échantillon à prédire soit très loin de tous les autres échantillons et que vous ayez pour chaque classe un score de 0. Dans ce cas-là, on ne trie pas les classes par score maximum, et on renvoit None à la place de la liste de classes triée.

In [ ]:
# A compléter
# Création de la classe de classifieur par fenêtres de Parzen
...

**2)** Testez deux modèles de Parzen (1 avec noyau uniforme, un avec noyau gaussien), sur le jeu de données 1, et avec comme paramètre h=8. Affichez les résultats (top1, top2 et matrice de confusion).

In [ ]:
# A compléter
# Tests des modèles de Parzen (uniforme, gaussien) sur le JDD 1, h=8
...

**3)** Un deuxième petit test mais cette fois avec un noyau uniforme et un paramètre h=1. Qu'est-ce qui ne va pas avec cette configuration ?

In [ ]:
# A compléter
# Le deuxième petit test
...

**_Réponse (analyse du problème) :_** 

**4)** A partir du problème que vous avez constaté, modifiez votre fonction de calcul de métrique pour prendre en charge les échantillons non classés. Un tel échantillon est donc considéré comme une fausse prédiction (d'un point de vue des scores top1 et top2). Pour ce qui est de la matrice de confusion, on rajoutera une colonne pour tous les échantillons non classés.

*Note :* Cette fonction pourra ensuite remplacer celle de base de la classe Modele (TP1)

In [ ]:
# A compléter
# Création de la classe de classifieur par fenêtres de Parzen, avec la correction sur la fonction de métriques
...

**5)** Re-testez maintenant votre modèle de Parzen uniforme avec h=1 sur le jeu de données 1. Vous devriez constater un nombre significatif de points non classés.

In [ ]:
# A compléter
# Création du modèle de Parzen uniforme, h=1
...

# Apprentissage du classifieur avec les données d'apprentissage
...

# Evaluation du classifieur avec les données d'évaluation
...

## Partie 3 : Cross-validation

Une question se pose : comment choisis-t-on le paramètre h ?

Une première solution serait de tester plusieurs paramètres h sur le jeu d'évaluation. Cette stratégie, c'est un **RED FLAG !!!** 
En fait, le jeu d'évaluation est uniquement destiné à l'évaluation d'un modèle : A aucun moment, il ne faut ajuster ces paramètres avec ce jeu, mais uniquement ce jeu d'apprentissage. Pourquoi cela ? Car nous souhaitons un modèle qui "généralise" bien, c'est-à-dire adapté à notre problème et non au jeu de données. Hors, en adaptant le ou les paramètre(s), on optimise le classifieur sur le jeu de données...

Alors, pour optimiser le paramètre h, on va utiliser le jeu d'apprentissage uniquement, et on va utiliser la cross validation. Le principe est simple : on va diviser le jeu d'apprentissage en N dossiers. Ensuite, pour chaque valeur de paramètre, on va faire N tests, avec pour chaque jeu un dossier de validation pour l'évaluation, et les autres dossiers pour l'apprentissage. On fera ensuite une moyenne des scores obtenus (ici, le top1).

**1)** Ecrivez la fonction de cross validation, qui prendra en entrée N (le nombre de dossiers crées pour la cross validation), les données et labels du jeu d'apprentissage, et la range des valeurs h à tester (h minimum, h maximum et step), ainsi que d'autres paramètres à définir. La fonction retournera la moyenne des scores top1 pour chaque valeur de h testée.

*Note :* Faites attention lors de la répartition des données en dossier. Il faut avoir des échantillons de chaque classe dans chaque dossier, sinon, ça va être compliqué...

In [ ]:
# A compléter
# Fonction de cross validation
...

**2)** Testez votre fonction de cross validation à 3 dossiers sur le jeu de données 1 avec un noyau gaussien. Quel est le meilleur h obtenu ? Et quel est la moyenne des top1 avec ce paramètre ?

*Note :* On souhaiterait avoir la meilleur valeur de h à une précision de 1 décimale après la virgule. Ayez une stratégie pour éviter de calculer toutes les valeurs possibles, car sinon, ça va être très long ;)

In [ ]:
# A compléter
# Cross validation sur le jeu de donnée 1, noyau uniforme
...

**3)** Faites de même maintenant avec les 3 jeux de données et les 2 types de noyau. Vous devriez donc avoir 6 valeurs de h optimisés et 6 scores obtenus sur les dossiers de validation. Remplissez ensuite le tableau avec tous ces résultats.

Pour comparaison, je vous donne mes résultats obtenus ;)

| Jeu de données / Noyau  | h | top1 |
|:-----------------------:|:-:|:----:|
| JDD 1 / Uniforme  | 4.2 | 100% |
| JDD 1 / Gaussien  | 0.1 | 100% |
| JDD 2 / Uniforme  | 5.2 | 94.99% |
| JDD 2 / Gaussien  | 1.8 | 94.99% |
| JDD 3 / Uniforme  | 4.1 | 74.79% |
| JDD 3 / Gaussien  | 2.5 | 75.19% |

In [ ]:
# A compléter
# Cross validation sur le jeu de donnée 1, noyau uniforme
...

| Jeu de données / Noyau  | h | top1 |
|:-----------------------:|:-:|:----:|
| JDD 1 / Uniforme  | 4.2 |  |
| JDD 1 / Gaussien  | 0.1 |  |
| JDD 2 / Uniforme  | 5.2 |  |
| JDD 2 / Gaussien  | 1.8 |  |
| JDD 3 / Uniforme  | 4.1 |  |
| JDD 3 / Gaussien  | 2.5 |  |

**4)** Evaluez maitenant vos classifieurs sur les jeux d'évaluation des 3 jeux de données, avec les 2 types de noyaux, et avec les paramètres h optimisés. Reportez les résultats sur le tableau suivant. Les résultats sont-ils en raccord avec ceux obtenus sur les jeux de validations ? Comparez les performances des classifieurs en dissociant l'analyse sur les 3 jeux de données.

In [ ]:
# A compléter
# Evaluation des classifieurs sur les jeux de tests
...

| Jeu de données / Noyau  | h | top1 valid | top1 eval |
|:-----------------------:|:-:|:----------:|:---------:|
| JDD 1 / Uniforme  | 4.2 |  |  |
| JDD 1 / Gaussien  | 0.1 |  |  |
| JDD 2 / Uniforme  | 5.2 |  |  |
| JDD 2 / Gaussien  | 1.8 |  |  |
| JDD 3 / Uniforme  | 4.1 |  |  |
| JDD 3 / Gaussien  | 2.5 |  |  |

**_Réponse :_**